In [ ]:
%matplotlib inline
import adaptive
import matplotlib.pyplot as plt
import pycqed as pq
import numpy as np
from pycqed.measurement import measurement_control
import pycqed.measurement.detector_functions as det
from qcodes import station
station = station.Station()


## Setting up the mock device

Measurements are controlled through the `MeasurementControl` usually instantiated as `MC`

In [ ]:
from pycqed.instrument_drivers.virtual_instruments.mock_device import Mock_Device
MC = measurement_control.MeasurementControl('MC',live_plot_enabled=True, verbose=True)
MC.station = station
station.add_component(MC)

mock_device = Mock_Device('mock_device')
mock_device.mw_pow(-20)
mock_device.res_freq(7.400023457e9)
mock_device.cw_noise_level(.0005)
mock_device.acq_delay(.05)


## Measuring a resonator using the conventional method
Points are chosen on a linspace of 100 points. This is enough to identify the location of the resonator. 

In [ ]:
freqs = np.linspace(7.39e9, 7.41e9, 100)

d = det.Function_Detector(mock_device.S21,value_names=['Magn', 'Phase'], 
                          value_units=['V', 'deg'])
MC.set_sweep_function(mock_device.mw_freq)
MC.set_sweep_points(freqs)
MC.set_detector_function(d)
dat=MC.run('test')

## Using 1D adaptive sampler from the MC 

This can also be done using an adaptive `Leaner1D` object, chosing 100 points optimally in the interval. 

In [ ]:
mock_device.acq_delay(.05)

In [ ]:

d = det.Function_Detector(mock_device.S21, value_names=['Magn', 'Phase'], value_units=['V', 'deg'])

MC.set_sweep_function(mock_device.mw_freq)
MC.set_detector_function(d)
MC.set_adaptive_function_parameters({'adaptive_function': adaptive.Learner1D, 
                                     'goal':lambda l: l.npoints>100, 
                                     'bounds':(7.39e9, 7.41e9)})
dat = MC.run(mode='adaptive')
from pycqed.analysis import measurement_analysis as ma
# ma.Homodyne_Analysis(close_fig=False, label='M')

## Two D learner

The learner can also be used to adaptively sample a 2D /heatmap type experiment. 
However, currently we do not have easy plotting function for that and we still need to rely on the adaptive Learner plotting methods. 

It would be great to have this working with a realtime pyqtgraph based plotting window so that we can use this without the notebooks. 

In [ ]:
d = det.Function_Detector(mock_device.S21, value_names=['Magn', 'Phase'], value_units=['V', 'deg'])
MC.set_sweep_function(mock_device.mw_freq)
MC.set_sweep_function_2D(mock_device.mw_pow)
MC.set_detector_function(d)
MC.set_adaptive_function_parameters({'adaptive_function': adaptive.Learner2D, 
                                     'goal':lambda l: l.npoints>20*20, 
                                     'bounds':((7.398e9, 7.402e9), 
                                               (-20, -10))})
dat = MC.run(mode='adaptive')



In [ ]:
# Required to be able to use the fancy interpolating plot 
adaptive.notebook_extension()
MC.learner.plot(tri_alpha=.1)